In [1]:
import pandas as pd
import iris
import xarray as xr
from eofs.xarray import Eof
import matplotlib.pyplot as plt

from eofs.multivariate.standard import MultivariateEof

Unable to open EPSG support file gcs.csv.  Try setting the GDAL_DATA environment variable to point to the directory containing EPSG csv files.


In [2]:
# "era", "ncep" or "jra"
rea_selection = "era"

# TODO maybe choose variable combination for validation later in this dict
rea_info = {
    "era": {'path':'C:/Users/Aspire_NB/Desktop/era5_data/small_era5_daily.nc',
            'vars':['r', 'q']},
}


try:
    ds_rea = xr.open_mfdataset(rea_info[rea_selection]['path'], decode_cf=True)
except KeyError:
    print("Choose a valid reanalysis from these options: {}".format(', '.join(rea_info.keys())))
    raise
except:
    print("Unexpected error")
    raise

print ('hallo')

hallo


In [4]:
# Prepare ERA   -------------------------------------------------------------------------------------
if rea_selection == 'era':
    ds_rea = xr.open_mfdataset("C:/Users/Aspire_NB/Desktop/era5_data/small_era5_daily.nc", decode_cf=True).chunk({'time':-1})
    ds_rea = ds_rea.rename({'latitude': 'lat', 'longitude': 'lon'})
pass

In [5]:
def calc_normalized_anomalies(ds_prep, window_size=21):
    """
    This method prepares the input Dataset. It calculates the normalized anomalies with means
    and std calculated with a centered window of size window_size
    """
    

    # Resample with mean over data if temporal resolution is higher than daily (Lower resolution not supported here)
    if pd.infer_freq(ds_prep.time.data) not in 'D':
        ds_prep = ds_prep.resample(time='1D').mean()
        ds_prep = ds_prep.chunk({'time':-1})

    # calculates the climatology and the standard deviation for further anomaly calculation
    ds_prep_roll = ds_prep.rolling(time=window_size, center=True).construct('window_dim') # creates the rolling window as a new dimension

    # mean after construct without dropna: first and last windows are considered, even if there are timesteps with missing values
    ds_prep_clim = ds_prep_roll.groupby('time.dayofyear').mean(dim=['window_dim','time'])

    # calculate standard deviation (dayofyear std) for rolling window over Target Day ± 10days (pool)
    ds_prep_std = ds_prep_roll.groupby('time.dayofyear').std(dim=xr.ALL_DIMS) # Calculates the std for dayofyear of TD + pool, shape(365,)

    # calculate daily normalized anomalies with mean and std from TD + pool
    ds_prep = ds_prep.groupby('time.dayofyear') - ds_prep_clim
    ds_rea = ds_prep.groupby('time.dayofyear') / ds_prep_std

    # Rechunking necessary after groupby
    ds_rea = ds_rea.chunk({'time': -1})

    return ds_rea

In [6]:
# Calculate normalized anomalies and construct rolled dimension
ds_rea = calc_normalized_anomalies(ds_rea)
ds_rea_roll = ds_rea.rolling(time=21, center=True).construct('window_dim')

C:\Users\Aspire_NB\Anaconda3\envs\Klimaenvironment\lib\site-packages\xarray\core\indexing.py:1195: PerformanceWarning: Slicing with an out-of-order index is generating 40 times more chunks
  return self.array[key]


In [7]:
ds_rea_roll

<xarray.Dataset>
Dimensions:    (lat: 3, lon: 9, time: 14610, window_dim: 21)
Coordinates:
  * lon        (lon) float32 9.0 10.0 11.0 12.0 13.0 14.0 15.0 16.0 17.0
  * lat        (lat) float32 46.0 47.0 48.0
  * time       (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2018-12-31
    dayofyear  (time) int64 dask.array<shape=(14610,), chunksize=(14610,)>
Dimensions without coordinates: window_dim
Data variables:
    r          (time, lat, lon, window_dim) float32 dask.array<shape=(14610, 3, 9, 21), chunksize=(14610, 3, 9, 21)>
    q          (time, lat, lon, window_dim) float32 dask.array<shape=(14610, 3, 9, 21), chunksize=(14610, 3, 9, 21)>
    msl        (time, lat, lon, window_dim) float32 dask.array<shape=(14610, 3, 9, 21), chunksize=(14610, 3, 9, 21)>

In [9]:
# Loop over dayofyear   ---------------------------------------------------------------------------------------------------------
for _, ds_test in ds_rea_roll.groupby("time.dayofyear"):

    # TODO
    ds_test = ds_test.rename({"time":"time_old"})
    #ds_test = ds_test.stack(time=('time_old', 'window_dim')).transpose('time_eof', 'lat', 'lon', 'nbnds')
    ds_test = ds_test.stack(time=('time_old', 'window_dim')).transpose('time', 'lat', 'lon')
    # Add attribute axis for EOF package to find new time dimension
    ds_test.coords['time'].attrs['axis'] = 'T'

    # Rename time dimension to avoid conflict if pcs are computed

    # Create an EOF solver to do the EOF analysis. Square-root of cosine of
    # latitude weights are applied before the computation of EOFs.
    # coslat = np.cos(np.deg2rad(z_djf.coords['latitude'].values)).clip(0., 1.)
    # wgts = np.sqrt(coslat)[..., np.newaxis]
    #solver = Eof(ds_test.pres.dropna('time'))
    
    msl = ds_test.msl.dropna('time').values
    r = ds_test.r.dropna('time').values
    q = ds_test.q.dropna('time').values
    
    msolver = MultivariateEof([msl, r, q])
    eofs_msl, eofs_r, eofs_q = msolver.eofs(neofs=5)
    pc = msolver.pcs(npcs=5)


    break

In [10]:
eofs_msl

array([[[ 0.1143795 ,  0.11237804,  0.11399539,  0.11347608,
          0.11468748,  0.1182808 ,  0.12362572,  0.12391073,
          0.12344657],
        [ 0.11374832,  0.11211598,  0.1132941 ,  0.11656632,
          0.1160461 ,  0.11926754,  0.12350215,  0.12575233,
          0.12530822],
        [ 0.12735403,  0.1288682 ,  0.12987606,  0.1310866 ,
          0.1313444 ,  0.13121511,  0.13071138,  0.12871194,
          0.12657952]],

       [[-0.14369449, -0.13782531, -0.14093941, -0.14645986,
         -0.14998285, -0.14622732, -0.13990182, -0.13856916,
         -0.13903807],
        [-0.12505369, -0.12333157, -0.12347782, -0.12675561,
         -0.12591317, -0.13050127, -0.13107961, -0.13286756,
         -0.13375942],
        [-0.12571701, -0.1259286 , -0.12458824, -0.12529089,
         -0.12627216, -0.12668452, -0.12711783, -0.12603831,
         -0.12960385]],

       [[ 0.0059174 ,  0.01430979,  0.00959483, -0.00420775,
         -0.00956721, -0.00134814,  0.01456717,  0.01528709,
    

In [11]:
eofs_r

array([[[-9.86091420e-02, -1.10496469e-01, -1.05361342e-01,
         -1.02739058e-01, -1.04757875e-01, -1.09079354e-01,
         -9.44225416e-02, -8.90793726e-02, -8.39906111e-02],
        [-1.21250309e-01, -1.19632743e-01, -1.21524096e-01,
         -1.17576525e-01, -1.14482388e-01, -1.04992330e-01,
         -9.57592651e-02, -9.05840844e-02, -8.93042237e-02],
        [-1.08449362e-01, -1.11468874e-01, -1.13146849e-01,
         -1.11189574e-01, -1.07297689e-01, -1.00445159e-01,
         -9.27638859e-02, -8.83959606e-02, -8.68111849e-02]],

       [[-1.44757302e-02, -1.69872921e-02, -1.75217185e-02,
         -1.39695127e-02, -1.89476982e-02, -1.19496686e-02,
         -5.89558948e-03, -5.29909227e-03,  1.64175857e-04],
        [ 1.26659218e-02,  3.45925824e-03, -6.79587899e-03,
         -1.10710226e-02, -1.53080896e-02, -3.27319950e-02,
         -2.86701191e-02, -2.10294221e-02, -1.25894342e-02],
        [-5.50859841e-03, -1.24291079e-02, -2.11982019e-02,
         -2.65385695e-02, -2.8145

In [12]:
eofs_q

array([[[-0.0997498 , -0.1123646 , -0.11138497, -0.11161389,
         -0.1164363 , -0.12044573, -0.11387041, -0.1112387 ,
         -0.10762055],
        [-0.10381856, -0.10742371, -0.11177521, -0.11425721,
         -0.11575379, -0.11137006, -0.10698597, -0.10404029,
         -0.10367522],
        [-0.09131648, -0.09621815, -0.10090466, -0.10307041,
         -0.10263192, -0.099935  , -0.09706165, -0.09606531,
         -0.09452391]],

       [[-0.12490538, -0.1293587 , -0.12744692, -0.12376702,
         -0.12520668, -0.12066114, -0.11448911, -0.11112451,
         -0.10100664],
        [-0.13748758, -0.1435731 , -0.14831395, -0.14922984,
         -0.14881262, -0.14600864, -0.13860816, -0.12972711,
         -0.11886726],
        [-0.14416116, -0.15206642, -0.1590317 , -0.16212712,
         -0.16185723, -0.15880696, -0.15295997, -0.14500692,
         -0.1308805 ]],

       [[-0.15744728, -0.14444052, -0.11249109, -0.08329777,
         -0.06349247, -0.04277432, -0.01416931, -0.00141306,
    